In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_with_lag/data_with_lag.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_3"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")


df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



          Umsatz  Temp_Very_Cold_current  Temp_Very_Cold_prev  \
3638  201.198426                       0                  0.0   
3639  265.261254                       0                  0.0   
3640  210.260241                       0                  0.0   
3641  190.686641                       0                  0.0   
3642  181.644870                       0                  0.0   
...          ...                     ...                  ...   
5452  333.774099                       0                  0.0   
5453  371.819770                       0                  0.0   
5454  433.820697                       0                  0.0   
5455  348.770346                       0                  0.0   
5456  285.872616                       0                  0.0   

      Temp_Very_Cold_next  Temp_Cold_current  Temp_Cold_prev  Temp_Cold_next  \
3638                  0.0                  0             0.0             0.0   
3639                  0.0                  0             0.

In [2]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Zeilen- und Spaltenanzeige in Pandas erweitern
pd.set_option("display.max_rows", None)  # Zeigt alle Zeilen
pd.set_option("display.max_columns", None)  # Zeigt alle Spalten

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                            1.000000
Sommer_prev                       0.635219
Sommer_current                    0.632812
Sommer_next                       0.629357
Monat_8_next                      0.481894
Monat_8_current                   0.467922
Monat_8_prev                      0.451121
Monat_7_prev                      0.444952
is_holiday_prev                   0.436179
is_holiday_next                   0.432808
Monat_7_current                   0.430227
is_holiday_current                0.429896
Monat_7_next                      0.414417
Temp_Warm_prev                    0.390744
Temp_Warm_current                 0.377216
Temp_Warm_next                    0.363280
wetter_sehr_schön_prev            0.299968
wetter_sehr_schön_next            0.286570
wetter_sehr_schön_current         0.285412
Number_of_ships_scaled_current    0.269484
Number_of_ships_current           0.269484
Ship_current                      0.264904
Samstag_prev                      0.246845
Temp_Hot_pr

In [3]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)



VPI_prev                          0.259874
VPI_next                          0.258091
VPI_current                       0.257948
Sommer_prev                       0.227995
Sommer_next                       0.227222
Sommer_current                    0.224967
Temp_Cold_current                 0.156736
Temp_Cold_next                    0.149344
Temp_Cold_prev                    0.145703
Winter_current                    0.128611
Winter_prev                       0.128182
Winter_next                       0.124382
Monat_8_next                      0.109353
Monat_8_current                   0.103448
Monat_8_prev                      0.100988
Monat_7_prev                      0.095725
Werktag_current                   0.093452
Monat_7_current                   0.091562
Temp_Warm_current                 0.089850
is_holiday_next                   0.088632
is_holiday_current                0.088035
is_holiday_prev                   0.084905
Monat_7_next                      0.082699
Temp_Warm_p

In [4]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Temp_Cold_current', 'Temp_Cold_prev', 'Temp_Cold_next', 'Cloud_Cloudy_current', 'Cloud_Cloudy_prev', 'Cloud_Cloudy_next', 'Wind_Light_current', 'Wind_Light_prev', 'Wind_Light_next', 'Wind_Moderate_current', 'Wind_Moderate_prev', 'Wind_Moderate_next', 'Wind_Strong_current', 'Wind_Strong_prev', 'Wind_Strong_next', 'KielerWoche_current', 'KielerWoche_prev', 'KielerWoche_next', 'Montag_current', 'Montag_prev', 'Montag_next', 'Dienstag_current', 'Dienstag_prev', 'Dienstag_next', 'Mittwoch_current', 'Mittwoch_prev', 'Mittwoch_next', 'Donnerstag_current', 'Donnerstag_prev', 'Donnerstag_next', 'Freitag_current', 'Freitag_prev', 'Freitag_next', 'Samstag_current', 'Samstag_prev', 'Samstag_next', 'Sonntag_current', 'Sonntag_prev', 'Sonntag_next', 'VPI_current', 'VPI_prev', 'VPI_next', 'Number_of_ships_current', 'Number_of_ships_prev', 'Number_of_ships_next', 'Ship_current', 'Ship_prev', 'Ship_next', 'is_holiday_current', 'is_holiday_prev', 'is_holiday_next', 'Weihnac

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                            Feature           VIF
0            Temp_Very_Cold_current           inf
1               Temp_Very_Cold_prev           inf
2               Temp_Very_Cold_next           inf
3                 Temp_Cold_current           inf
4                    Temp_Cold_prev  2.445569e+07
5                    Temp_Cold_next           inf
6                 Temp_Mild_current           inf
7                    Temp_Mild_prev           inf
8                    Temp_Mild_next           inf
9                 Temp_Warm_current           inf
10                   Temp_Warm_prev           inf
11                   Temp_Warm_next           inf
12                 Temp_Hot_current           inf
13                    Temp_Hot_prev           inf
14                    Temp_Hot_next           inf
15              Cloud_Clear_current           inf
16                 Cloud_Clear_prev           inf
17                 Cloud_Clear_next           inf
18      Cloud_Partly_Cloudy_current           inf


In [6]:
# suche miteinander korrelierrende features

In [7]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('Herbst_next', 'Herbst_current'), ('Monat_12_current', 'Winter_current'), ('Sommer_current', 'Temp_Cold_next'), ('Donnerstag_next', 'Dienstag_prev'), ('Monat_1_current', 'Winter_next'), ('Wind_ok_prev', 'Wind_Strong_prev'), ('Monat_2_next', 'Monat_2_current'), ('Monat_4_current', 'Frühling_current'), ('Monat_2_next', 'Monat_2_prev'), ('Temp_Mild_next', 'Temp_Cold_next'), ('Monat_11_next', 'Herbst_current'), ('Sommer_prev', 'Temp_Cold_current'), ('Number_of_ships_scaled_prev', 'Number_of_ships_prev'), ('Sommer_current', 'Temp_Warm_current'), ('Monat_12_prev', 'Weihnachtsmarkt_next'), ('Weather_Light_Issues_prev', 'Weather_Light_Issues_current'), ('Monat_1_prev', 'Winter_next'), ('Monat_4_prev', 'Frühling_current'), ('Monat_6_current', 'KielerWoche_next'), ('Temp_Hot_prev', 'Temp_Hot_current'), ('Monat_3_prev', 'Frühling_current'), ('Monat_6_current', 'KielerWoche_current'), ('Monat_4_current', 'Markt_current'), ('Monat_3_next', 'Frühling_prev'), ('Sonn